## Generating C code for the right-hand-sides of Einstein's equations of general relativity in the [BSSN](http://www2.yukawa.kyoto-u.ac.jp/~yuichiro.sekiguchi/3+1.pdf) formalism, in ***curvilinear*** coordinates, using a covariant reference metric approach

### ***Citations***: Generic curvilinear coordinate reference metric approach matches that of [Ruchlin, Etienne, and Baumgarte (2018)](https://arxiv.org/abs/1712.07658), which is an extension of the spherical coordinate reference metric approach of [Baumgarte, Montero, Cordero-Carrión, and Müller (2012)](https://arxiv.org/abs/1211.6632), which builds upon the covariant "Lagrangian" BSSN formalism of [Brown (2009)](https://arxiv.org/abs/0902.3652). *See also citations within each article.*

**Background**: NRPy+'s original purpose was to be an easy-to-use code capable of generating Einstein's equations in a broad class of [singular](https://en.wikipedia.org/wiki/Coordinate_singularity), curvilinear coordinate systems, where the user need only input the scale factors of the underlying reference metric. Upon generating these equations, NRPy+ would then leverage SymPy's [common-expression-elimination (CSE)](https://en.wikipedia.org/wiki/Common_subexpression_elimination) and C code generation routines, coupled to its own [single-instruction, multiple-data (SIMD)](https://en.wikipedia.org/wiki/SIMD) functions, to generate highly-optimized C code.

This tutorial module demonstrates how Einstein's equations of general relativity in this formulation are constructed and output within NRPy+. As Einstein's equations in this formalism take the form of highly nonlinear, coupled *wave equations*, the [tutorial module on the scalar wave equation in curvilinear coordinates](Tutorial-ScalarWaveCurvilinear.ipynb) is *required* reading before beginning this module. That module, as well as its own prerequisite [module on reference metrics within NRPy+](Tutorial-Reference_Metric.ipynb) provides the needed overview of how NRPy+ handles reference metrics.

In summary, the equations we wish to input into NRPy+ are as follows (Eqs. 11 and 12 in [Ruchlin, Etienne, and Baumgarte (2018)](https://arxiv.org/abs/1712.07658)):

\begin{align}
  \partial_{\perp} \varepsilon_{i j} {} = {} & \frac{2}{3} \bar{\gamma}_{i j} \left (\alpha \bar{A}_{k}^{k} - \bar{D}_{k} \beta^{k}\right ) + 2 \hat{D}_{(i} \beta_{j)} - 2 \alpha \bar{A}_{i j} \; , \\
  \partial_{\perp} \bar{A}_{i j} {} = {} & -\frac{2}{3} \bar{A}_{i j} \bar{D}_{k} \beta^{k} - 2 \alpha \bar{A}_{i k} {\bar{A}^{k}}_{j} + \alpha \bar{A}_{i j} K \nonumber \\
  & + e^{-4 \phi} \left \{-2 \alpha \bar{D}_{i} \bar{D}_{j} \phi + 4 \alpha \bar{D}_{i} \phi \bar{D}_{j} \phi \right . \nonumber \\
    & \left . + 4 \bar{D}_{(i} \alpha \bar{D}_{j)} \phi - \bar{D}_{i} \bar{D}_{j} \alpha + \alpha \bar{R}_{i j} \right \}^{\text{TF}} \; , \\
  \partial_{\perp} W {} = {} & -\frac{1}{3} W \left (\bar{D}_{k} \beta^{k} - \alpha K \right ) \; , \\
  \partial_{\perp} K {} = {} & \frac{1}{3} \alpha K^{2} + \alpha \bar{A}_{i j} \bar{A}^{i j} \nonumber \\
  & - e^{-4 \phi} \left (\bar{D}_{i} \bar{D}^{i} \alpha + 2 \bar{D}^{i} \alpha \bar{D}_{i} \phi \right ) \; , \\
  \partial_{\perp} \bar{\Lambda}^{i} {} = {} & \bar{\gamma}^{j k} \hat{D}_{j} \hat{D}_{k} \beta^{i} + \frac{2}{3} \Delta^{i} \bar{D}_{j} \beta^{j} + \frac{1}{3} \bar{D}^{i} \bar{D}_{j} \beta^{j} \nonumber \\
  & - 2 \bar{A}^{i j} \left (\partial_{j} \alpha - 6 \partial_{j} \phi \right ) + 2 \bar{A}^{j k} \Delta_{j k}^{i} \nonumber \\
  & -\frac{4}{3} \alpha \bar{\gamma}^{i j} \partial_{j} K \; .
\end{align}

where the $\text{TF}$ superscript denotes the trace-free part. Also, $\bar{R}_{ij}$ is the conformal Ricci tensor, computed via
\begin{align}
  \bar{R}_{i j} {} = {} & - \frac{1}{2} \bar{\gamma}^{k l} \hat{D}_{k} \hat{D}_{l} \bar{\gamma}_{i j} + \bar{\gamma}_{k(i} \hat{D}_{j)} \bar{\Lambda}^{k} + \Delta^{k} \Delta_{(i j) k} \nonumber \\
  & + \bar{\gamma}^{k l} \left (2 \Delta_{k(i}^{m} \Delta_{j) m l} + \Delta_{i k}^{m} \Delta_{m j l} \right ) \; .
\end{align}

In [1]:
# Step 1: import all needed modules from NRPy+:
import NRPy_param_funcs as par
import indexedexp as ixp
import grid as gri
import finite_difference as fin
import reference_metric as rfm
from outputC import *

In [2]:
# Step 2a: Initialize BSSN_RHS parameters

# Step 2b: Set desired spatial dimension
par.set_paramsvals_value("grid::DIM = 3")

# Step 3: Register all needed *evolved* gridfunctions.
# Step 3a: Register indexed quantities, using ixp.register_... functions
hDD = ixp.register_gridfunctions_for_single_rank2("EVOL","hDD", "sym12")
aDD = ixp.register_gridfunctions_for_single_rank2("EVOL","aDD", "sym12")
lambdaU = ixp.register_gridfunctions_for_single_rank1("EVOL","lambdaU")
vetU = ixp.register_gridfunctions_for_single_rank1("EVOL","vetU")
betU = ixp.register_gridfunctions_for_single_rank1("EVOL","betU")
# Step 3b: Register scalar quantities, using gri.register_gridfunctions()
trK, cf, alpha = gri.register_gridfunctions("EVOL",["trK","cf","alpha"])

# Step 4: Register all *auxiliary* gridfunctions.
# Step 4a: Register indexed quantities, using ixp.register_... functions
RbarDD = ixp.register_gridfunctions_for_single_rank2("EVOL","RbarDD", "sym12")
# Step 4b: Register scalar quantities, using gri.register_gridfunctions()
detg = gri.register_gridfunctions("AUX",["detg"])

In [3]:
# Step 4: Define 